In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
import numpy as np
import pandas as pd
import torch
import torchvision
from torch import optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from sklearn.metrics import pairwise_distances
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tqdm.notebook import tqdm
import os
import sys
import time
import torch.nn as nn
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from stream_generators.market_basket_dataset import MarketBasketDataset, BinaryEncodingTransform, RemoveItemsTransform
from utils.MatplotlibUtils import reduce_dims_and_plot
from network.auto_encoder import AutoEncoder
from losses.knn_loss import KNNLoss
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from soft_decision_tree.sdt_model import SDT
from sklearn.metrics import davies_bouldin_score

In [2]:
k = 16
tree_depth = 6
device = 'cuda'
dataset_path = r"/mnt/qnap/ekosman/Groceries_dataset.csv"

## Create the market basket dataset and use one-hot encoding for items

In [3]:
dataset = MarketBasketDataset(dataset_path=dataset_path)

In [4]:
model = AutoEncoder(dataset.n_items, 50, 4).train().to(device)
epochs = 500
lr = 5e-3
batch_size = 512
log_every = 5

In [5]:
dataset.transform = torchvision.transforms.Compose([
    RemoveItemsTransform(p=0.5),
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [6]:
model.train()
data_iter = torch.utils.data.DataLoader(dataset,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     num_workers=1,
                                     pin_memory=True)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, factor=0.7, threshold=1e-4)
knn_crt = KNNLoss(k=k).to(device)
losses = []
alpha = 10/170
gamma = 2
for epoch in range(epochs):
    total_loss = 0
    for iteration, (batch, target) in enumerate(data_iter):
        batch = batch.to(device)
        target = target.to(device)
        outputs, iterm = model(batch, return_intermidiate=True)
        mse_loss = F.binary_cross_entropy_with_logits(outputs, target, reduction='none')
        mask = torch.ones_like(mse_loss)
        mask[target == 0] = alpha ** gamma
        mask[target == 1] = (1 - alpha) ** gamma
        mse_loss = (mse_loss * mask).sum(dim=-1).mean()
        try:
            knn_loss = knn_crt(iterm)
            if torch.isinf(knn_loss):
                knn_loss = 0
        except ValueError:
            knn_loss = torch.tensor(0)
        loss = mse_loss + knn_loss
        total_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if iteration % log_every == 0:
            print(f"Epoch {epoch} / {epochs} | iteration {iteration} / {len(data_iter)} | Total Loss: {loss.item()} | KNN Loss: {knn_loss.item()} | BCE Loss: {mse_loss.item()}")
    
    scheduler.step(total_loss / (iteration + 1))
    losses.append(total_loss / (iteration + 1))

Epoch 0 / 500 | iteration 0 / 30 | Total Loss: 8.210041999816895 | KNN Loss: 6.227721214294434 | BCE Loss: 1.98232102394104
Epoch 0 / 500 | iteration 5 / 30 | Total Loss: 8.172706604003906 | KNN Loss: 6.227598190307617 | BCE Loss: 1.9451082944869995
Epoch 0 / 500 | iteration 10 / 30 | Total Loss: 8.17885684967041 | KNN Loss: 6.227121353149414 | BCE Loss: 1.9517356157302856
Epoch 0 / 500 | iteration 15 / 30 | Total Loss: 8.124919891357422 | KNN Loss: 6.22711181640625 | BCE Loss: 1.8978075981140137
Epoch 0 / 500 | iteration 20 / 30 | Total Loss: 8.122896194458008 | KNN Loss: 6.22642707824707 | BCE Loss: 1.896469235420227
Epoch 0 / 500 | iteration 25 / 30 | Total Loss: 8.110965728759766 | KNN Loss: 6.225857257843018 | BCE Loss: 1.8851089477539062
Epoch 1 / 500 | iteration 0 / 30 | Total Loss: 8.14926528930664 | KNN Loss: 6.225307464599609 | BCE Loss: 1.9239575862884521
Epoch 1 / 500 | iteration 5 / 30 | Total Loss: 8.112594604492188 | KNN Loss: 6.2245941162109375 | BCE Loss: 1.88800001144

Epoch 10 / 500 | iteration 25 / 30 | Total Loss: 6.615598201751709 | KNN Loss: 5.525787353515625 | BCE Loss: 1.089810848236084
Epoch 11 / 500 | iteration 0 / 30 | Total Loss: 6.5095672607421875 | KNN Loss: 5.394506454467773 | BCE Loss: 1.115060567855835
Epoch 11 / 500 | iteration 5 / 30 | Total Loss: 6.456127166748047 | KNN Loss: 5.326240062713623 | BCE Loss: 1.1298872232437134
Epoch 11 / 500 | iteration 10 / 30 | Total Loss: 6.258234977722168 | KNN Loss: 5.168399810791016 | BCE Loss: 1.0898349285125732
Epoch 11 / 500 | iteration 15 / 30 | Total Loss: 6.153439521789551 | KNN Loss: 5.073712348937988 | BCE Loss: 1.0797271728515625
Epoch 11 / 500 | iteration 20 / 30 | Total Loss: 6.1178364753723145 | KNN Loss: 4.985212326049805 | BCE Loss: 1.1326241493225098
Epoch 11 / 500 | iteration 25 / 30 | Total Loss: 5.943134307861328 | KNN Loss: 4.838218688964844 | BCE Loss: 1.1049153804779053
Epoch 12 / 500 | iteration 0 / 30 | Total Loss: 5.827757835388184 | KNN Loss: 4.71121072769165 | BCE Loss:

Epoch 21 / 500 | iteration 20 / 30 | Total Loss: 4.275885581970215 | KNN Loss: 3.237459421157837 | BCE Loss: 1.0384260416030884
Epoch 21 / 500 | iteration 25 / 30 | Total Loss: 4.278428554534912 | KNN Loss: 3.2170639038085938 | BCE Loss: 1.0613646507263184
Epoch 22 / 500 | iteration 0 / 30 | Total Loss: 4.236767768859863 | KNN Loss: 3.1986160278320312 | BCE Loss: 1.0381519794464111
Epoch 22 / 500 | iteration 5 / 30 | Total Loss: 4.295625686645508 | KNN Loss: 3.219217300415039 | BCE Loss: 1.0764086246490479
Epoch 22 / 500 | iteration 10 / 30 | Total Loss: 4.2434821128845215 | KNN Loss: 3.205167531967163 | BCE Loss: 1.038314700126648
Epoch 22 / 500 | iteration 15 / 30 | Total Loss: 4.253411293029785 | KNN Loss: 3.2115750312805176 | BCE Loss: 1.0418360233306885
Epoch 22 / 500 | iteration 20 / 30 | Total Loss: 4.2888031005859375 | KNN Loss: 3.2246108055114746 | BCE Loss: 1.064192533493042
Epoch 22 / 500 | iteration 25 / 30 | Total Loss: 4.273770332336426 | KNN Loss: 3.2208034992218018 | BC

Epoch 32 / 500 | iteration 15 / 30 | Total Loss: 4.146391868591309 | KNN Loss: 3.1157655715942383 | BCE Loss: 1.0306265354156494
Epoch 32 / 500 | iteration 20 / 30 | Total Loss: 4.150697231292725 | KNN Loss: 3.132648468017578 | BCE Loss: 1.0180487632751465
Epoch 32 / 500 | iteration 25 / 30 | Total Loss: 4.123737335205078 | KNN Loss: 3.1169633865356445 | BCE Loss: 1.0067739486694336
Epoch 33 / 500 | iteration 0 / 30 | Total Loss: 4.14653205871582 | KNN Loss: 3.1173195838928223 | BCE Loss: 1.0292123556137085
Epoch 33 / 500 | iteration 5 / 30 | Total Loss: 4.153446197509766 | KNN Loss: 3.1160006523132324 | BCE Loss: 1.0374457836151123
Epoch 33 / 500 | iteration 10 / 30 | Total Loss: 4.164685249328613 | KNN Loss: 3.132547616958618 | BCE Loss: 1.0321377515792847
Epoch 33 / 500 | iteration 15 / 30 | Total Loss: 4.202447414398193 | KNN Loss: 3.1936850547790527 | BCE Loss: 1.0087623596191406
Epoch 33 / 500 | iteration 20 / 30 | Total Loss: 4.221055507659912 | KNN Loss: 3.13733172416687 | BCE 

Epoch 43 / 500 | iteration 5 / 30 | Total Loss: 4.191071510314941 | KNN Loss: 3.13553786277771 | BCE Loss: 1.055533528327942
Epoch 43 / 500 | iteration 10 / 30 | Total Loss: 4.151248931884766 | KNN Loss: 3.1207659244537354 | BCE Loss: 1.0304827690124512
Epoch 43 / 500 | iteration 15 / 30 | Total Loss: 4.1442718505859375 | KNN Loss: 3.1346371173858643 | BCE Loss: 1.0096344947814941
Epoch 43 / 500 | iteration 20 / 30 | Total Loss: 4.177558898925781 | KNN Loss: 3.1473333835601807 | BCE Loss: 1.0302252769470215
Epoch 43 / 500 | iteration 25 / 30 | Total Loss: 4.125032901763916 | KNN Loss: 3.108682632446289 | BCE Loss: 1.016350269317627
Epoch 44 / 500 | iteration 0 / 30 | Total Loss: 4.140707015991211 | KNN Loss: 3.1019270420074463 | BCE Loss: 1.0387797355651855
Epoch 44 / 500 | iteration 5 / 30 | Total Loss: 4.171140193939209 | KNN Loss: 3.1368513107299805 | BCE Loss: 1.0342888832092285
Epoch 44 / 500 | iteration 10 / 30 | Total Loss: 4.199906349182129 | KNN Loss: 3.154616594314575 | BCE L

...

Epoch 448 / 500 | iteration 0 / 30 | Total Loss: 4.125553131103516 | KNN Loss: 3.0818722248077393 | BCE Loss: 1.0436809062957764
Epoch 448 / 500 | iteration 5 / 30 | Total Loss: 4.056644439697266 | KNN Loss: 3.0316333770751953 | BCE Loss: 1.0250113010406494
Epoch 448 / 500 | iteration 10 / 30 | Total Loss: 4.055647373199463 | KNN Loss: 3.04300594329834 | BCE Loss: 1.012641429901123
Epoch 448 / 500 | iteration 15 / 30 | Total Loss: 4.073719024658203 | KNN Loss: 3.0572991371154785 | BCE Loss: 1.0164196491241455
Epoch 448 / 500 | iteration 20 / 30 | Total Loss: 4.058377265930176 | KNN Loss: 3.069915533065796 | BCE Loss: 0.9884615540504456
Epoch 448 / 500 | iteration 25 / 30 | Total Loss: 4.1179118156433105 | KNN Loss: 3.0713958740234375 | BCE Loss: 1.0465160608291626
Epoch 449 / 500 | iteration 0 / 30 | Total Loss: 4.121257305145264 | KNN Loss: 3.0971975326538086 | BCE Loss: 1.024059772491455
Epoch 449 / 500 | iteration 5 / 30 | Total Loss: 4.09030818939209 | KNN Loss: 3.055854558944702 |

Epoch 458 / 500 | iteration 20 / 30 | Total Loss: 4.085227966308594 | KNN Loss: 3.048189640045166 | BCE Loss: 1.0370380878448486
Epoch 458 / 500 | iteration 25 / 30 | Total Loss: 4.100359916687012 | KNN Loss: 3.0805132389068604 | BCE Loss: 1.019846796989441
Epoch 459 / 500 | iteration 0 / 30 | Total Loss: 4.06060791015625 | KNN Loss: 3.0455482006073 | BCE Loss: 1.0150599479675293
Epoch 459 / 500 | iteration 5 / 30 | Total Loss: 4.077927589416504 | KNN Loss: 3.0534939765930176 | BCE Loss: 1.0244338512420654
Epoch 459 / 500 | iteration 10 / 30 | Total Loss: 4.124232769012451 | KNN Loss: 3.058230400085449 | BCE Loss: 1.066002368927002
Epoch 459 / 500 | iteration 15 / 30 | Total Loss: 4.094308376312256 | KNN Loss: 3.0752480030059814 | BCE Loss: 1.0190603733062744
Epoch 459 / 500 | iteration 20 / 30 | Total Loss: 4.099851608276367 | KNN Loss: 3.10188627243042 | BCE Loss: 0.9979653358459473
Epoch 459 / 500 | iteration 25 / 30 | Total Loss: 4.0859375 | KNN Loss: 3.0637705326080322 | BCE Loss:

Epoch 469 / 500 | iteration 10 / 30 | Total Loss: 4.0941572189331055 | KNN Loss: 3.043483257293701 | BCE Loss: 1.0506737232208252
Epoch 469 / 500 | iteration 15 / 30 | Total Loss: 4.117823123931885 | KNN Loss: 3.0962131023406982 | BCE Loss: 1.021609902381897
Epoch 469 / 500 | iteration 20 / 30 | Total Loss: 4.04673957824707 | KNN Loss: 3.030496120452881 | BCE Loss: 1.016243577003479
Epoch 469 / 500 | iteration 25 / 30 | Total Loss: 4.1146559715271 | KNN Loss: 3.081186532974243 | BCE Loss: 1.033469557762146
Epoch 470 / 500 | iteration 0 / 30 | Total Loss: 4.117110252380371 | KNN Loss: 3.0767993927001953 | BCE Loss: 1.0403110980987549
Epoch 470 / 500 | iteration 5 / 30 | Total Loss: 4.091004371643066 | KNN Loss: 3.0672128200531006 | BCE Loss: 1.023791790008545
Epoch 470 / 500 | iteration 10 / 30 | Total Loss: 4.059557914733887 | KNN Loss: 3.0564918518066406 | BCE Loss: 1.0030661821365356
Epoch 470 / 500 | iteration 15 / 30 | Total Loss: 4.03196907043457 | KNN Loss: 3.050567388534546 | BC

Epoch 480 / 500 | iteration 0 / 30 | Total Loss: 4.090553283691406 | KNN Loss: 3.075035810470581 | BCE Loss: 1.0155173540115356
Epoch 480 / 500 | iteration 5 / 30 | Total Loss: 4.072437286376953 | KNN Loss: 3.0551910400390625 | BCE Loss: 1.0172460079193115
Epoch 480 / 500 | iteration 10 / 30 | Total Loss: 4.121167182922363 | KNN Loss: 3.074995994567871 | BCE Loss: 1.0461711883544922
Epoch 480 / 500 | iteration 15 / 30 | Total Loss: 4.090211868286133 | KNN Loss: 3.0612542629241943 | BCE Loss: 1.0289576053619385
Epoch 480 / 500 | iteration 20 / 30 | Total Loss: 4.082388877868652 | KNN Loss: 3.0901005268096924 | BCE Loss: 0.9922885894775391
Epoch 480 / 500 | iteration 25 / 30 | Total Loss: 4.121117115020752 | KNN Loss: 3.07875657081604 | BCE Loss: 1.0423604249954224
Epoch 481 / 500 | iteration 0 / 30 | Total Loss: 4.074170112609863 | KNN Loss: 3.0565381050109863 | BCE Loss: 1.017632246017456
Epoch 481 / 500 | iteration 5 / 30 | Total Loss: 4.0654191970825195 | KNN Loss: 3.064542531967163 

Epoch 490 / 500 | iteration 20 / 30 | Total Loss: 4.043079376220703 | KNN Loss: 3.027883291244507 | BCE Loss: 1.0151962041854858
Epoch 490 / 500 | iteration 25 / 30 | Total Loss: 4.08083438873291 | KNN Loss: 3.0618417263031006 | BCE Loss: 1.01899254322052
Epoch 491 / 500 | iteration 0 / 30 | Total Loss: 4.067022323608398 | KNN Loss: 3.072608470916748 | BCE Loss: 0.9944136738777161
Epoch 491 / 500 | iteration 5 / 30 | Total Loss: 4.042782783508301 | KNN Loss: 3.047851324081421 | BCE Loss: 0.9949314594268799
Epoch 491 / 500 | iteration 10 / 30 | Total Loss: 4.111721038818359 | KNN Loss: 3.0793263912200928 | BCE Loss: 1.0323948860168457
Epoch 491 / 500 | iteration 15 / 30 | Total Loss: 4.1432647705078125 | KNN Loss: 3.1072874069213867 | BCE Loss: 1.0359772443771362
Epoch 491 / 500 | iteration 20 / 30 | Total Loss: 4.0979437828063965 | KNN Loss: 3.1002767086029053 | BCE Loss: 0.9976669549942017
Epoch 491 / 500 | iteration 25 / 30 | Total Loss: 4.115687370300293 | KNN Loss: 3.07952213287353

In [7]:
outputs, iterm = model(dataset[67][0].unsqueeze(0).to(device), return_intermidiate=True)
print(outputs)
print(outputs.min())
print(outputs.max())

tensor([[ 3.1004e+00,  2.8424e+00,  2.8250e+00,  3.0073e+00,  2.9105e+00,
          6.3167e-01,  2.4144e+00,  2.4124e+00,  2.2677e+00,  1.7566e+00,
          2.4531e+00,  2.0375e+00,  9.6963e-01,  1.9713e+00,  1.1873e+00,
          1.7947e+00,  2.8798e+00,  2.7177e+00,  2.5930e+00,  2.2776e+00,
          1.6449e+00,  3.1229e+00,  2.5383e+00,  2.7050e+00,  2.7791e+00,
          1.9863e+00,  2.1137e+00,  1.4696e+00,  1.4666e+00,  4.0106e-01,
         -2.4083e-01,  1.1589e+00,  3.1289e-01,  1.0797e+00,  1.6853e+00,
          1.5168e+00,  7.8847e-01,  3.2470e+00,  6.9854e-01,  1.3243e+00,
          1.1100e+00, -5.2067e-01, -3.9265e-02,  2.4637e+00,  2.3308e+00,
          6.6560e-01, -1.2188e-02,  2.0878e-01,  1.6554e+00,  2.5232e+00,
          1.8229e+00,  1.0992e-01,  1.2048e+00,  6.4537e-01, -4.4819e-01,
          1.3678e+00,  1.6303e+00,  1.3435e+00,  1.4171e+00,  2.0011e+00,
          8.6431e-01,  8.2060e-01,  2.8669e-01,  1.8782e+00,  1.3908e+00,
          1.7580e+00, -1.9129e+00,  3.

In [8]:
plt.figure()
plt.plot(losses)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
dataset.transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)
dataset.target_transform = torchvision.transforms.Compose([
    BinaryEncodingTransform(mapping=dataset.items_to_idx),
]
)

In [10]:
dataset_ = [d[0].to('cpu') for d in dataset]

In [11]:
model = model.eval().to('cpu')
projections = model.calculate_intermidiate(dataset_)

100%|██████████| 15/15 [00:00<00:00, 102.32it/s]


In [12]:
distances = pairwise_distances(projections)
# distances = np.triu(distances)
distances_f = distances.flatten()

plt.matshow(distances)
plt.colorbar()
plt.figure()
plt.hist(distances_f[distances_f > 0], bins=1000)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Fit DBSCAN and calculate indices


In [13]:
clusters = DBSCAN(eps=0.2, min_samples=80).fit_predict(projections)
# scores = []
# best_score = float('inf')
# clusters = None
# range_ = list(range(5, 20))
# for k in tqdm(range_):
#     y = GaussianMixture(n_components=k).fit_predict(projections)
#     cur_score = davies_bouldin_score(projections, y)
#     scores.append(cur_score)
    
#     if cur_score < best_score:
#         best_score = cur_score
#         clusters = y

In [14]:
perplexity = 100

p = reduce_dims_and_plot(projections[clusters != -1],
                         y=clusters[clusters != -1],
                         title=f'perplexity: {perplexity}',
                         file_name=None,
                         perplexity=perplexity,
                         library='Multicore-TSNE',
                         perform_PCA=False,
                         projected=None,
                         figure_type='2d',
                         show_figure=True,
                         close_figure=False,
                         text=None)    

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
# from sklearn.tree import DecisionTreeClassifier
# from sklearn import tree
# from sklearn.tree import _tree

In [16]:
tensor_dataset = torch.stack(dataset_)

In [17]:
# clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=5)
# clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
# print(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
# print(clf.get_depth())

In [18]:
# scores = []
# for min_samples in range(1,50, 1):
#     clf = DecisionTreeClassifier(max_depth=200, min_samples_leaf=min_samples)
#     clf = clf.fit(tensor_dataset[clusters!=-1], clusters[clusters != -1])
#     scores.append(clf.score(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
    
# plt.figure()
# plt.plot(list(range(1,50, 1)), scores)
# plt.show()

In [19]:
def get_rules(tree, feature_names, class_names):
    tree_ = tree.tree_
    feature_name = [
        feature_names[i] if i != _tree.TREE_UNDEFINED else "undefined!"
        for i in tree_.feature
    ]

    paths = []
    path = []
    
    def recurse(node, path, paths):
        if tree_.feature[node] != _tree.TREE_UNDEFINED:
            name = feature_name[node]
            threshold = tree_.threshold[node]
            p1, p2 = list(path), list(path)
#             p1 += [f"({name} <= {np.round(threshold, 3)})"]
            p1 += [(name, '<=', np.round(threshold, 3))]
            recurse(tree_.children_left[node], p1, paths)
            p2 += [(name, '>', np.round(threshold, 3))]
            recurse(tree_.children_right[node], p2, paths)
        else:
            path += [(tree_.value[node], tree_.n_node_samples[node])]
            paths += [path]
            
    recurse(0, path, paths)

    # sort by samples count
    samples_count = [p[-1][1] for p in paths]
    ii = list(np.argsort(samples_count))
    paths = [paths[i] for i in reversed(ii)]
    
    rules = []
    for path in paths:
        rule = []
        
        for p in path[:-1]:
            rule += [p]
        target = " then "
        if class_names is None:
            target += "response: "+str(np.round(path[-1][0][0][0],3))
        else:
            classes = path[-1][0][0]
            l = np.argmax(classes)
            target += f"class: {class_names[l]} (proba: {np.round(100.0*classes[l]/np.sum(classes),2)}%)"
           
        proba = np.round(100.0*classes[l]/np.sum(classes),2)
        target += f" | based on {path[-1][1]:,} samples"
        rule_wrapper = {'target': target, 'rule': rule, 'proba': proba}
        rules += [rule_wrapper]
        
    return rules

In [20]:
# rules = get_rules(clf, dataset.items, clusters[clusters != -1])

# for rule in rules:
#     n_pos = 0
#     for c,p,v in rule['rule']:
#         if p == '>':
#             n_pos += 1
#     rule['pos'] = n_pos

In [21]:
# plt.figure()
# probs = [r['proba'] for r in rules]
# plt.hist(probs, bins = 100)
# plt.show()

In [22]:
# rules = sorted(rules, key=lambda x:x['pos'])
# rules = [r for r in rules if r['proba'] > 50]
# print(len(rules))

In [23]:
# for i in range(17):
#     r_i = rules[i]
#     print(f"------------- rule {i} length {len(r_i)} -------------")
#     print(r_i)

# Train a Soft-Decision-Tree given the self-labels

## Prepare the dataset

In [24]:
tree_dataset = list(zip(tensor_dataset[clusters!=-1], clusters[clusters != -1]))
batch_size = 512
tree_loader = torch.utils.data.DataLoader(tree_dataset, batch_size=batch_size, shuffle=True)

# Define how we prune the weights of a node

In [25]:
def prune_node(node_weights, factor=1):
    w = node_weights.cpu().detach().numpy()
    mean_ = np.mean(w)
    std_ = np.std(w)
    node_weights[((mean_ - std_ * factor) < node_weights) & (node_weights < (mean_ + std_ * factor))] = 0
    return node_weights

def prune_node_keep(node_weights, keep=4):
    w = node_weights.cpu().detach().numpy()
    throw_idx = np.argsort(abs(w))[:-keep]
    node_weights[throw_idx] = 0
    return node_weights

def prune_tree(tree_, factor):
    new_weights = tree_.inner_nodes.weight.clone()
    for i in range(new_weights.shape[0]):
        res = prune_node_keep(new_weights[i, :], factor)
        new_weights[i, :] = res

    with torch.no_grad():
        tree_.inner_nodes.weight.copy_(new_weights)
        
def sparseness(x):
    s = []
    for i in range(x.shape[0]):
        x_ = x[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        s.append(sp)
    return np.mean(s)

def compute_regularization_by_level(tree):
    total_reg = 0
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_level = np.floor(np.log2(i+1))
        node_reg = torch.norm(tree.inner_nodes.weight[i].view(-1), 2)
        total_reg += 2**(-cur_level) * node_reg
    return total_reg

def show_sparseness(tree):
    avg_sp = sparseness(tree.inner_nodes.weight)
    print(f"Average sparseness: {avg_sp}")
    layer = 0
    sps = []
    for i in range(tree.inner_nodes.weight.shape[0]):
        cur_layer = int(np.floor(np.log2(i+1)))
        if cur_layer != layer:
            print(f"layer {layer}: {np.mean(sps)}")
            sps = []
            layer = cur_layer

        x_ = tree.inner_nodes.weight[i, :]
        sp = (len(x_) - torch.norm(x_, 0).item()) / len(x_)
        sps.append(sp)
        
    return avg_sp

## Training configurations

In [26]:
def do_epoch(model, loader, device, log_interval, losses, accs, epoch, iteration):
    model = model.train()
    for batch_idx, (data, target) in enumerate(loader):
        iteration += 1
        data, target = data.to(device), target.to(device)

        output, penalty = tree.forward(data)

        # Loss
        loss_tree = criterion(output, target.view(-1))

        # Penalty
        loss_tree += penalty

        # Sparse regularization
#         fc_params = torch.cat([x.view(-1) for x in tree.inner_nodes.parameters()])
#         regularization = sparsity_lamda * torch.norm(fc_params, 2)
        regularization = sparsity_lamda * compute_regularization_by_level(tree)
        loss = loss_tree

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())

        pred = output.data.max(1)[1]
        correct = pred.eq(target.view(-1).data).sum()
        accs.append(correct.item() / data.size()[0])

        # Print training status
        if batch_idx % log_interval == 0:
            print(f"Epoch: {epoch:02d} | Batch: {batch_idx:03d} / {len(loader):03d} | Total loss: {loss.item():.3f} | Reg loss: {regularization.item():.3f} | Tree loss: {loss_tree.item():.3f} | Accuracy: {correct.item() / data.size()[0]:03f} | {round((time.time() - start_time) / iteration, 3)} sec/iter")
            
    return iteration


In [27]:
lr = 5e-3
weight_decay = 5e-4
sparsity_lamda = 2e-3
epochs = 100
output_dim = len(set(clusters))
log_interval = 1
use_cuda = device != 'cpu'

In [28]:
tree = SDT(input_dim=tensor_dataset.shape[1], output_dim=len(clusters - 1), depth=tree_depth, lamda=1e-3, use_cuda=use_cuda)
optimizer = torch.optim.Adam(tree.parameters(),
                                 lr=lr,
                                 weight_decay=weight_decay)
criterion = nn.CrossEntropyLoss()
tree = tree.to(device)

In [29]:
losses = []
accs = []
sparsity = []

In [30]:
start_time = time.time()
iteration = 0
for epoch in range(epochs):
    # Training
    avg_sp = show_sparseness(tree)
    sparsity.append(avg_sp)
    iteration = do_epoch(tree, tree_loader, device, log_interval, losses, accs, epoch, iteration)
    
    if epoch % 1 == 0:
        prune_tree(tree, factor=3)
        

Average sparseness: 0.0
layer 0: 0.0
layer 1: 0.0
layer 2: 0.0
layer 3: 0.0
layer 4: 0.0
Epoch: 00 | Batch: 000 / 016 | Total loss: 9.646 | Reg loss: 0.007 | Tree loss: 9.646 | Accuracy: 0.000000 | 0.108 sec/iter
Epoch: 00 | Batch: 001 / 016 | Total loss: 9.650 | Reg loss: 0.007 | Tree loss: 9.650 | Accuracy: 0.000000 | 0.09 sec/iter
Epoch: 00 | Batch: 002 / 016 | Total loss: 9.630 | Reg loss: 0.007 | Tree loss: 9.630 | Accuracy: 0.000000 | 0.081 sec/iter
Epoch: 00 | Batch: 003 / 016 | Total loss: 9.629 | Reg loss: 0.006 | Tree loss: 9.629 | Accuracy: 0.000000 | 0.078 sec/iter
Epoch: 00 | Batch: 004 / 016 | Total loss: 9.625 | Reg loss: 0.006 | Tree loss: 9.625 | Accuracy: 0.000000 | 0.074 sec/iter
Epoch: 00 | Batch: 005 / 016 | Total loss: 9.605 | Reg loss: 0.006 | Tree loss: 9.605 | Accuracy: 0.000000 | 0.071 sec/iter
Epoch: 00 | Batch: 006 / 016 | Total loss: 9.603 | Reg loss: 0.006 | Tree loss: 9.603 | Accuracy: 0.000000 | 0.069 sec/iter
Epoch: 00 | Batch: 007 / 016 | Total loss: 9

Epoch: 03 | Batch: 014 / 016 | Total loss: 9.360 | Reg loss: 0.007 | Tree loss: 9.360 | Accuracy: 0.115234 | 0.058 sec/iter
Epoch: 03 | Batch: 015 / 016 | Total loss: 9.366 | Reg loss: 0.007 | Tree loss: 9.366 | Accuracy: 0.106838 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 04 | Batch: 000 / 016 | Total loss: 9.391 | Reg loss: 0.005 | Tree loss: 9.391 | Accuracy: 0.091797 | 0.058 sec/iter
Epoch: 04 | Batch: 001 / 016 | Total loss: 9.373 | Reg loss: 0.005 | Tree loss: 9.373 | Accuracy: 0.125000 | 0.058 sec/iter
Epoch: 04 | Batch: 002 / 016 | Total loss: 9.383 | Reg loss: 0.005 | Tree loss: 9.383 | Accuracy: 0.091797 | 0.058 sec/iter
Epoch: 04 | Batch: 003 / 016 | Total loss: 9.364 | Reg loss: 0.005 | Tree loss: 9.364 | Accuracy: 0.132812 | 0.058 sec/iter
Epoch: 04 | Batch: 004 / 016 | Total loss: 9.367 | Reg loss: 0.006 | Tree los

Epoch: 07 | Batch: 013 / 016 | Total loss: 9.084 | Reg loss: 0.011 | Tree loss: 9.084 | Accuracy: 0.087891 | 0.058 sec/iter
Epoch: 07 | Batch: 014 / 016 | Total loss: 9.072 | Reg loss: 0.011 | Tree loss: 9.072 | Accuracy: 0.121094 | 0.058 sec/iter
Epoch: 07 | Batch: 015 / 016 | Total loss: 9.063 | Reg loss: 0.011 | Tree loss: 9.063 | Accuracy: 0.123932 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 08 | Batch: 000 / 016 | Total loss: 9.107 | Reg loss: 0.009 | Tree loss: 9.107 | Accuracy: 0.121094 | 0.058 sec/iter
Epoch: 08 | Batch: 001 / 016 | Total loss: 9.108 | Reg loss: 0.009 | Tree loss: 9.108 | Accuracy: 0.119141 | 0.059 sec/iter
Epoch: 08 | Batch: 002 / 016 | Total loss: 9.108 | Reg loss: 0.010 | Tree loss: 9.108 | Accuracy: 0.111328 | 0.059 sec/iter
Epoch: 08 | Batch: 003 / 016 | Total loss: 9.089 | Reg loss: 0.010 | Tree los

Epoch: 11 | Batch: 010 / 016 | Total loss: 8.765 | Reg loss: 0.014 | Tree loss: 8.765 | Accuracy: 0.091797 | 0.058 sec/iter
Epoch: 11 | Batch: 011 / 016 | Total loss: 8.719 | Reg loss: 0.015 | Tree loss: 8.719 | Accuracy: 0.132812 | 0.058 sec/iter
Epoch: 11 | Batch: 012 / 016 | Total loss: 8.702 | Reg loss: 0.015 | Tree loss: 8.702 | Accuracy: 0.113281 | 0.058 sec/iter
Epoch: 11 | Batch: 013 / 016 | Total loss: 8.717 | Reg loss: 0.015 | Tree loss: 8.717 | Accuracy: 0.113281 | 0.058 sec/iter
Epoch: 11 | Batch: 014 / 016 | Total loss: 8.704 | Reg loss: 0.015 | Tree loss: 8.704 | Accuracy: 0.117188 | 0.058 sec/iter
Epoch: 11 | Batch: 015 / 016 | Total loss: 8.673 | Reg loss: 0.016 | Tree loss: 8.673 | Accuracy: 0.121795 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 12 | Batch: 000 / 016 | Total loss: 8.786 | Reg loss: 0.014 | Tree los

Epoch: 15 | Batch: 010 / 016 | Total loss: 8.314 | Reg loss: 0.018 | Tree loss: 8.314 | Accuracy: 0.083984 | 0.058 sec/iter
Epoch: 15 | Batch: 011 / 016 | Total loss: 8.309 | Reg loss: 0.018 | Tree loss: 8.309 | Accuracy: 0.125000 | 0.058 sec/iter
Epoch: 15 | Batch: 012 / 016 | Total loss: 8.258 | Reg loss: 0.018 | Tree loss: 8.258 | Accuracy: 0.109375 | 0.058 sec/iter
Epoch: 15 | Batch: 013 / 016 | Total loss: 8.287 | Reg loss: 0.018 | Tree loss: 8.287 | Accuracy: 0.097656 | 0.058 sec/iter
Epoch: 15 | Batch: 014 / 016 | Total loss: 8.257 | Reg loss: 0.018 | Tree loss: 8.257 | Accuracy: 0.095703 | 0.058 sec/iter
Epoch: 15 | Batch: 015 / 016 | Total loss: 8.222 | Reg loss: 0.019 | Tree loss: 8.222 | Accuracy: 0.117521 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 16 | Batch: 000 / 016 | Total loss: 8.400 | Reg loss: 0.017 | Tree los

...

Epoch: 83 | Batch: 001 / 016 | Total loss: 4.320 | Reg loss: 0.032 | Tree loss: 4.320 | Accuracy: 0.107422 | 0.058 sec/iter
Epoch: 83 | Batch: 002 / 016 | Total loss: 4.347 | Reg loss: 0.032 | Tree loss: 4.347 | Accuracy: 0.109375 | 0.058 sec/iter
Epoch: 83 | Batch: 003 / 016 | Total loss: 4.353 | Reg loss: 0.032 | Tree loss: 4.353 | Accuracy: 0.107422 | 0.058 sec/iter
Epoch: 83 | Batch: 004 / 016 | Total loss: 4.281 | Reg loss: 0.032 | Tree loss: 4.281 | Accuracy: 0.125000 | 0.058 sec/iter
Epoch: 83 | Batch: 005 / 016 | Total loss: 4.320 | Reg loss: 0.032 | Tree loss: 4.320 | Accuracy: 0.113281 | 0.058 sec/iter
Epoch: 83 | Batch: 006 / 016 | Total loss: 4.321 | Reg loss: 0.032 | Tree loss: 4.321 | Accuracy: 0.097656 | 0.058 sec/iter
Epoch: 83 | Batch: 007 / 016 | Total loss: 4.226 | Reg loss: 0.032 | Tree loss: 4.226 | Accuracy: 0.125000 | 0.058 sec/iter
Epoch: 83 | Batch: 008 / 016 | Total loss: 4.352 | Reg loss: 0.032 | Tree loss: 4.352 | Accuracy: 0.097656 | 0.058 sec/iter
Epoch: 8

Epoch: 86 | Batch: 015 / 016 | Total loss: 4.110 | Reg loss: 0.032 | Tree loss: 4.110 | Accuracy: 0.119658 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 87 | Batch: 000 / 016 | Total loss: 4.260 | Reg loss: 0.032 | Tree loss: 4.260 | Accuracy: 0.113281 | 0.058 sec/iter
Epoch: 87 | Batch: 001 / 016 | Total loss: 4.387 | Reg loss: 0.032 | Tree loss: 4.387 | Accuracy: 0.083984 | 0.058 sec/iter
Epoch: 87 | Batch: 002 / 016 | Total loss: 4.282 | Reg loss: 0.032 | Tree loss: 4.282 | Accuracy: 0.132812 | 0.058 sec/iter
Epoch: 87 | Batch: 003 / 016 | Total loss: 4.293 | Reg loss: 0.032 | Tree loss: 4.293 | Accuracy: 0.126953 | 0.058 sec/iter
Epoch: 87 | Batch: 004 / 016 | Total loss: 4.365 | Reg loss: 0.032 | Tree loss: 4.365 | Accuracy: 0.105469 | 0.058 sec/iter
Epoch: 87 | Batch: 005 / 016 | Total loss: 4.278 | Reg loss: 0.032 | Tree los

Epoch: 90 | Batch: 015 / 016 | Total loss: 4.153 | Reg loss: 0.032 | Tree loss: 4.153 | Accuracy: 0.102564 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 91 | Batch: 000 / 016 | Total loss: 4.259 | Reg loss: 0.032 | Tree loss: 4.259 | Accuracy: 0.119141 | 0.058 sec/iter
Epoch: 91 | Batch: 001 / 016 | Total loss: 4.310 | Reg loss: 0.032 | Tree loss: 4.310 | Accuracy: 0.097656 | 0.058 sec/iter
Epoch: 91 | Batch: 002 / 016 | Total loss: 4.260 | Reg loss: 0.032 | Tree loss: 4.260 | Accuracy: 0.093750 | 0.058 sec/iter
Epoch: 91 | Batch: 003 / 016 | Total loss: 4.216 | Reg loss: 0.032 | Tree loss: 4.216 | Accuracy: 0.125000 | 0.058 sec/iter
Epoch: 91 | Batch: 004 / 016 | Total loss: 4.278 | Reg loss: 0.032 | Tree loss: 4.278 | Accuracy: 0.085938 | 0.058 sec/iter
Epoch: 91 | Batch: 005 / 016 | Total loss: 4.231 | Reg loss: 0.032 | Tree los

Epoch: 94 | Batch: 015 / 016 | Total loss: 4.160 | Reg loss: 0.032 | Tree loss: 4.160 | Accuracy: 0.094017 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 95 | Batch: 000 / 016 | Total loss: 4.283 | Reg loss: 0.032 | Tree loss: 4.283 | Accuracy: 0.105469 | 0.058 sec/iter
Epoch: 95 | Batch: 001 / 016 | Total loss: 4.199 | Reg loss: 0.032 | Tree loss: 4.199 | Accuracy: 0.103516 | 0.058 sec/iter
Epoch: 95 | Batch: 002 / 016 | Total loss: 4.191 | Reg loss: 0.032 | Tree loss: 4.191 | Accuracy: 0.115234 | 0.058 sec/iter
Epoch: 95 | Batch: 003 / 016 | Total loss: 4.202 | Reg loss: 0.032 | Tree loss: 4.202 | Accuracy: 0.107422 | 0.058 sec/iter
Epoch: 95 | Batch: 004 / 016 | Total loss: 4.152 | Reg loss: 0.032 | Tree loss: 4.152 | Accuracy: 0.109375 | 0.058 sec/iter
Epoch: 95 | Batch: 005 / 016 | Total loss: 4.247 | Reg loss: 0.032 | Tree los

Epoch: 98 | Batch: 013 / 016 | Total loss: 3.986 | Reg loss: 0.032 | Tree loss: 3.986 | Accuracy: 0.123047 | 0.058 sec/iter
Epoch: 98 | Batch: 014 / 016 | Total loss: 4.051 | Reg loss: 0.032 | Tree loss: 4.051 | Accuracy: 0.105469 | 0.058 sec/iter
Epoch: 98 | Batch: 015 / 016 | Total loss: 4.014 | Reg loss: 0.032 | Tree loss: 4.014 | Accuracy: 0.094017 | 0.058 sec/iter
Average sparseness: 0.9821428571428567
layer 0: 0.9821428571428571
layer 1: 0.9821428571428571
layer 2: 0.9821428571428571
layer 3: 0.9821428571428571
layer 4: 0.9821428571428571
Epoch: 99 | Batch: 000 / 016 | Total loss: 4.145 | Reg loss: 0.032 | Tree loss: 4.145 | Accuracy: 0.119141 | 0.058 sec/iter
Epoch: 99 | Batch: 001 / 016 | Total loss: 4.189 | Reg loss: 0.032 | Tree loss: 4.189 | Accuracy: 0.089844 | 0.058 sec/iter
Epoch: 99 | Batch: 002 / 016 | Total loss: 4.146 | Reg loss: 0.032 | Tree loss: 4.146 | Accuracy: 0.113281 | 0.058 sec/iter
Epoch: 99 | Batch: 003 / 016 | Total loss: 4.172 | Reg loss: 0.032 | Tree los

In [31]:
plt.figure(figsize=(10, 5))
plt.ylabel("Accuracy")
plt.xlabel('Iteration')
plt.plot(accs, label='Accuracy vs iteration')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel('Iteration')
plt.plot(losses, label='Loss vs iteration')
plt.yscale("log")
plt.show()

plt.figure()
weights = tree.inner_nodes.weight.cpu().detach().numpy().flatten()
plt.hist(weights, bins=500)
weights_std = np.std(weights)
weights_mean = np.mean(weights)
plt.axvline(weights_mean + weights_std, color='r')
plt.axvline(weights_mean - weights_std, color='r')
plt.title(f"Mean: {weights_mean}   |   STD: {weights_std}")
plt.yscale("log")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Tree Visualization

In [33]:
plt.figure(figsize=(15, 10), dpi=80)
avg_height, root = tree.visualize()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Average height: 5.892857142857143


# Extract Rules

# Accumulate samples in the leaves

In [34]:
print(f"Number of patterns: {len(root.get_leaves())}")

Number of patterns: 56


In [35]:
method = 'greedy'

In [36]:
root.clear_leaves_samples()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(tree_loader):
        root.accumulate_samples(data, method)

/home/eitan.k/.local/lib/python3.6/site-packages/torch/nn/functional.py:1709: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


# Tighten boundaries

In [37]:
attr_names = dataset.items

# print(attr_names)
leaves = root.get_leaves()
sum_comprehensibility = 0
comprehensibilities = []
for pattern_counter, leaf in enumerate(leaves):
    leaf.reset_path()
    leaf.tighten_with_accumulated_samples()
    conds = leaf.get_path_conditions(attr_names)
    print(f"============== Pattern {pattern_counter + 1} ==============")
    comprehensibilities.append(sum([cond.comprehensibility for cond in conds]))
    
print(f"Average comprehensibility: {np.mean(comprehensibilities)}")
print(f"std comprehensibility: {np.std(comprehensibilities)}")
print(f"var comprehensibility: {np.var(comprehensibilities)}")
print(f"minimum comprehensibility: {np.min(comprehensibilities)}")
print(f"maximum comprehensibility: {np.max(comprehensibilities)}")

============== Pattern 1 ==============
============== Pattern 2 ==============
============== Pattern 3 ==============
============== Pattern 4 ==============
============== Pattern 5 ==============
============== Pattern 6 ==============
============== Pattern 7 ==============
============== Pattern 8 ==============
============== Pattern 9 ==============
============== Pattern 10 ==============
============== Pattern 11 ==============
============== Pattern 12 ==============
============== Pattern 13 ==============
============== Pattern 14 ==============
============== Pattern 15 ==============
============== Pattern 16 ==============
============== Pattern 17 ==============
============== Pattern 18 ==============
============== Pattern 19 ==============
============== Pattern 20 ==============
============== Pattern 21 ==============
============== Pattern 22 ==============
============== Pattern 23 ==============
============== Pattern 24 ==============
============== Pattern 25

/home/eitan.k/EntangledExplainableClustering/soft_decision_tree/sdt_model.py:14: RuntimeWarning: divide by zero encountered in log
  return np.log(1 / (1 - x))
